In [17]:
# imports
import os
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr
from dotenv import load_dotenv
import json
from pydub import AudioSegment
from pydub.playback import play
import base64
from io import BytesIO
from PIL import Image
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time
from bs4 import BeautifulSoup
import requests
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
from IPython.display import Audio, display
import pytesseract
import numpy as np
import PyPDF2
from openai import ChatCompletion


In [18]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [19]:
openai = OpenAI()

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
hf_token = os.getenv("HF_TOKEN")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if hf_token:
    print("HF_TOKEN API key exists")


OpenAI API Key exists and begins sk-proj-


In [20]:
system_prompt = (
    "You are an assistant designed to help customers analyze and compare their bills. "
    "When provided with two images of bills, respond briefly by identifying key pricing differences, highlighting increases, decreases, or unchanged amounts. "
    "If a customer uploads an image that is not a bill, respond politely and inform them that only bill images are supported for analysis. "
    "Additionally, provide a specific reasons for the changes in the bill, such as: "
    "'This month bill increase was due to a Netflix subscription, which does not appear on the previous bill, a charge for an extra line ending in ###, and a roaming charge for a call made to France.'"
)


In [21]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    with open(pdf_file.name, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        return " ".join(page.extract_text() for page in reader.pages)

# Function to extract text from image
def extract_text_from_image(image_file):
    image = Image.open(image_file)
    return pytesseract.image_to_string(image)

# Main function to handle file uploads and comparison
def compare_files(file1, file2):
    # Determine file types and extract text
    if file1.name.endswith('.pdf') and file2.name.endswith('.pdf'):
        text1 = extract_text_from_pdf(file1)
        text2 = extract_text_from_pdf(file2)
    elif file1.name.lower().endswith(('png', 'jpg', 'jpeg')) and file2.name.lower().endswith(('png', 'jpg', 'jpeg')):
        text1 = extract_text_from_image(file1)
        text2 = extract_text_from_image(file2)
    else:
        return "Error: Please upload two files of the same type (both PDFs or both images)."

    # Use OpenAI API to analyze the differences
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Compare the following bills. The key differences between {text1} and {text2} are:\\ and this is why:"}
        ]
    )
    return response.choices[0].message.content
   

In [22]:
# Gradio interface
interface = gr.Interface(
    fn=compare_files,
    inputs=[
        gr.File(type="filepath", label="Upload First File (PDF or Image)"),
        gr.File(type="filepath", label="Upload Second File (PDF or Image)")
    ],
    outputs="text",
    title="File Comparison Tool",
    description="Upload two PDF files or two image files (PNG/JPG) to compare their content."
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://5e825e2bffd486b43b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
